In [ ]:
%matplotlib widget
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import IPython
import numpy as np
from ipywidgets import interact

# Modulacije

Ideja modulacija je da spakuju signali sa nekom nižom frekvencijom u signal sa značajno višom frekvencijom. Tada je moguće da se prenose radio talasima ili optikom. Pre se to radilo analogno, sa npr radio stanicama i FM prijemnicima, dok se danas to uglavnom radi digitalno.

# Amplitudska Modulacija

AM je jedna od prvih korišćenih modulacija (kada stariji kažu dugi talasi) i ona je veoma jednostavna za implementaciju. Njena mana je što nije toliko otporna na šum.

Implementira se tako što se amplituda signala visoke frekvencije smanjuje ili povećava, srazmerno sa ulaznim modulišućim signalom.

In [ ]:
fs = 200e3
tmax = 10e-3

f_signal = 1e3

ts = np.linspace(0, tmax, int(tmax * fs))

modulator = np.sin(2 * np.pi * f_signal * ts)

In [ ]:
Ac = 1 #Amplituda nosioca
Am = 0.5 #Amplituda modulišućeg signala

fc = 10e3

carrier = np.sin(2 * np.pi * fc * ts)

In [ ]:
carrier_AM = (Ac + Am * modulator) * carrier
amplitude = np.abs(signal.hilbert(carrier_AM))

In [ ]:
fig, ((ax1, ax4), (ax2, ax5), (ax3, ax6)) = plt.subplots(3, 2, sharex = "col", figsize=(10, 6), sharey="col")
ax1.plot(ts, modulator)
ax2.plot(ts, carrier)
ax3.plot(ts, carrier_AM)
ax3.plot(ts, amplitude);

ax4.magnitude_spectrum(modulator, Fs=fs, scale='dB');
ax5.magnitude_spectrum(carrier, Fs=fs, scale='dB');
ax6.magnitude_spectrum(carrier_AM, Fs=fs, scale='dB');

In [ ]:
plt.figure()
plt.specgram(carrier_AM, Fs=fs, scale="linear", NFFT = 64, noverlap=32)
plt.ylim(5e3, 15e3)

Ovakav signal je onda moguće odašiljati pomoću antene u etar.

Demodulacija AM signala se može izvršiti na više načina. Prvi je merenje envelope, i tako su radili prvi demodulatori (diodni detektori).

<center><img src="dataset/images/envelope-detector.png"/></center>

Takodje je moguće demodulisati signal množenjem sa nemodulisanim nosećim signalom (product detector).

<center><img src="dataset/images/product-detector.png"/></center>

In [ ]:
def simulateDiodeDetector(x, R, C, fs):
    output = np.zeros_like(x)
    t = 0
    Vc = x[0]
    output[0] = x[0]
    
    dt = 1 / fs
    
    for i in range(1, len(x)):
        Vc = Vc * np.exp(- dt / (R * C))
        if x[i] > Vc:
            Vc = x[i]

        output[i] = Vc
        
    return output

diodeDetected = simulateDiodeDetector(carrier_AM, 10e3, 0.047e-6, fs)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.plot(ts, carrier_AM)
ax1.plot(ts, amplitude)
ax1.plot(ts, diodeDetected);

ax2.magnitude_spectrum(carrier_AM, Fs=fs, scale='dB');
ax2.magnitude_spectrum(amplitude, Fs=fs, scale='dB');
ax2.magnitude_spectrum(diodeDetected, Fs=fs, scale='dB');

Kako zvučnici imaju pretežnu Band-Pass karakteristiku, ovaj signal se može razaznati.

In [ ]:
audioLowPassFilter = signal.butter(20, 20000, fs=fs)

In [ ]:
speakerModel = signal.ellip(6, 0.1, 20, (2000, 10000), 'bandpass', fs=fs)

In [ ]:
filteredDiodeDetected_audio = signal.lfilter(audioLowPassFilter[0], audioLowPassFilter[1], diodeDetected)
filteredDiodeDetected = signal.lfilter(speakerModel[0], speakerModel[1], filteredDiodeDetected_audio)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = "col", figsize=(10, 4), sharey="col")
ax1.plot(ts, modulator)
ax1_2 = ax1.twinx()
ax1_2.plot(ts, filteredDiodeDetected, c="orange");

ax2.magnitude_spectrum(modulator, Fs=fs, scale='dB');
ax2.magnitude_spectrum(filteredDiodeDetected, Fs=fs, scale='dB');

In [ ]:
productDemodulated = carrier_AM * carrier

In [ ]:
fig, ((ax1, ax4), (ax2, ax5), (ax3, ax6)) = plt.subplots(3, 2, sharex = "col", figsize=(10, 6), sharey="col")
ax1.plot(ts, carrier_AM)
ax1.plot(ts, amplitude);
ax2.plot(ts, carrier)
ax3.plot(ts, productDemodulated)

ax4.magnitude_spectrum(carrier_AM, Fs=fs, scale='dB');
ax5.magnitude_spectrum(carrier, Fs=fs, scale='dB');
ax6.magnitude_spectrum(productDemodulated, Fs=fs, scale='dB');

In [ ]:
filteredProductDemodulated_audio = signal.lfilter(audioLowPassFilter[0], audioLowPassFilter[1], productDemodulated)
filteredProductDemodulated = signal.lfilter(speakerModel[0], speakerModel[1], filteredProductDemodulated_audio)

In [ ]:
fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(2, 2, sharex = "col", figsize=(10, 6))
ax1.plot(ts, productDemodulated)
ax1.plot(ts, amplitude);

ax2.plot(ts, filteredDiodeDetected, c="orange");

ax2.plot(ts, filteredProductDemodulated)


ax3.magnitude_spectrum(productDemodulated, Fs=fs, scale='dB');
ax4.magnitude_spectrum(modulator, Fs=fs, scale='dB');
ax4.magnitude_spectrum(filteredProductDemodulated, Fs=fs, scale='dB');
